In [2]:
# 想直接看对自己模型的提升的，把max_model改成20，赋权重融合到自己模型里
# 按我的经验，融合之前先算一下差异性,np.corr,可以达到0.97左右的相关性的话，融合效果会比较好

# 想用的就用，不想用的没必要在这浪费时间
# 也呼吁大家多看看鱼的代码，里面干货很多，这份代码的成绩取决于你放进来什么特征
# 这份特征在我自己特征工程之后可以达到榜上的成绩，原始特征大概是88左右

import h2o
from h2o.automl import H2OAutoML
h2o.init(max_mem_size='64G')

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11.0.1+13-LTS, mixed mode)
  Starting server from c:\python37\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\LIUCUI~1.LON\AppData\Local\Temp\tmpugowmdhm
  JVM stdout: C:\Users\LIUCUI~1.LON\AppData\Local\Temp\tmpugowmdhm\h2o_liucuili_started_from_python.out
  JVM stderr: C:\Users\LIUCUI~1.LON\AppData\Local\Temp\tmpugowmdhm\h2o_liucuili_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Asia/Shanghai
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.1
H2O cluster version age:,12 days
H2O cluster name:,H2O_from_python_liucuili_9n67js
H2O cluster total nodes:,1
H2O cluster free memory:,64 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [31]:
train = h2o.upload_file("./datasets1/jinnan_round1_train_20181227.csv", header=0)
test = h2o.upload_file("./datasets1/jinnan_round1_testA_20181227.csv", header=0)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [20]:
train.head()

id,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,A17,A18,A19,A20,A21,A22,A23,A24,A25,A26,A27,A28,B1,B2,B3,B4,B5,B6,B7,B8,B9,B10,B11,B12,B13,B14,rate
sample_1,300,nan,405,700,1970-01-01 06:30:00,25,,nan,1970-01-01 08:00:00,100,1970-01-01 09:00:00,103,0.2,1970-01-01 10:00:00,104,1970-01-01 11:00:00,105,0.2,300,15:00-15:30,40,9,5,1970-01-01 15:30:00,70,1970-01-01 16:00:00,78,0:00-1:00,320,3.5,3.5,1:00-2:30,1970-01-01 02:30:00,60,1970-01-01 04:30:00,40,8:00-9:30,10:00-11:30,,800,0.15,400,0.9
sample_100,300,nan,405,700,1970-01-01 20:00:00,29,,nan,1970-01-01 22:00:00,101,1970-01-01 23:00:00,103,0.2,1970-01-01 00:00:00,104,1970-01-01 01:00:00,105,0.2,200,1:00-2:00,50,9,5,1970-01-01 02:00:00,80,1970-01-01 03:00:00,73,3:00-4:00,320,3.5,3.5,4:00-5:00,1970-01-01 05:00:00,80,1970-01-01 07:00:00,45,7:00-8:30,8:30-10:00,10:00-11:00,1200,0.15,400,0.902
sample_1000,300,nan,405,700,1970-01-01 17:00:00,30,,nan,1970-01-01 18:00:00,100,1970-01-01 19:00:00,101,0.2,1970-01-01 20:00:00,103,1970-01-01 21:00:00,105,0.2,200,21:30-22:00,50,10,5,1970-01-01 22:30:00,70,1970-01-01 23:30:00,78,3:30-4:30,290,3.5,3.5,4:30-5:30,1970-01-01 05:30:00,65,1970-01-01 07:30:00,45,9:00-10:30,10:30-12:00,,800,0.15,400,0.902
sample_1001,300,nan,405,700,1970-01-01 01:00:00,21,,nan,1970-01-01 02:30:00,100,1970-01-01 03:30:00,101,0.2,1970-01-01 04:30:00,103,1970-01-01 05:30:00,105,0.2,200,6:00-6:30,50,9,5,1970-01-01 07:00:00,70,1970-01-01 07:30:00,78,10:30-11:00,290,3.5,3.5,11:00-12:00,1970-01-01 12:00:00,60,1970-01-01 19:00:00,36,19:30-21:00,21:40-23:10,,800,0.15,380,0.868
sample_1002,300,nan,405,700,1970-01-01 14:00:00,21,,nan,1970-01-01 15:00:00,100,1970-01-01 16:00:00,101,0.2,1970-01-01 17:00:00,102,1970-01-01 18:00:00,103,0.2,200,18:00-18:30,50,9,5,1970-01-01 18:30:00,70,1970-01-01 19:00:00,78,19:00-20:00,300,3.5,3.5,20:00-21:30,1970-01-01 21:30:00,60,1970-01-01 04:00:00,39,4:30-6:00,7:00-8:00,,800,0.15,380,0.878
sample_1004,300,nan,405,700,1970-01-01 07:00:00,20,,nan,1970-01-01 08:30:00,100,1970-01-01 09:30:00,102,0.2,1970-01-01 10:30:00,103,1970-01-01 11:30:00,105,0.2,200,12:00-12:30,50,9,5,1970-01-01 13:00:00,70,1970-01-01 13:30:00,78,17:00-18:00,290,3.5,3.5,18:00-19:00,1970-01-01 19:00:00,60,1970-01-01 21:00:00,40,23:30-1:00,2:00-3:30,,800,0.15,400,0.902
sample_1005,300,nan,405,700,1970-01-01 17:30:00,21,,nan,1970-01-01 18:30:00,100,1970-01-01 19:30:00,101,0.2,1970-01-01 20:30:00,103,1970-01-01 21:30:00,105,0.2,200,21:30-22:00,40,9,5,1970-01-01 22:00:00,70,1970-01-01 22:30:00,78,2:00-3:00,300,3.5,3.5,3:00-4:30,1970-01-01 04:30:00,60,1970-01-01 06:30:00,40,9:00-10:30,11:00-12:30,,800,0.15,400,0.902
sample_1006,300,nan,405,700,1970-01-01 01:30:00,35,,nan,1970-01-01 02:30:00,100,1970-01-01 03:30:00,101,0.2,1970-01-01 04:30:00,103,1970-01-01 05:30:00,105,0.2,200,6:00-6:30,50,9,5,1970-01-01 06:30:00,70,1970-01-01 07:00:00,78,11:00-12:00,280,3.5,3.5,12:00-13:30,1970-01-01 13:30:00,60,1970-01-01 19:00:00,40,19:00-20:30,21:30-23:00,,800,0.15,400,0.902
sample_1007,300,nan,405,700,1970-01-01 12:00:00,35,,nan,1970-01-01 13:00:00,100,1970-01-01 14:00:00,102,0.2,1970-01-01 15:00:00,103,1970-01-01 16:00:00,105,0.2,200,16:30-17:00,50,10,5,1970-01-01 17:30:00,70,1970-01-01 18:00:00,70,22:30-23:30,290,3.5,3.5,23:30-0:30,1970-01-01 00:30:00,65,1970-01-01 03:00:00,45,5:00-6:30,7:30-9:00,,800,0.15,400,0.913
sample_1008,300,nan,405,700,1970-01-01 22:00:00,21,,nan,1970-01-01 23:30:00,100,1970-01-01 00:30:00,101,0.2,1970-01-01 01:30:00,103,1970-01-01 02:30:00,104,0.2,200,3:00-3:30,50,9,5,1970-01-01 04:00:00,70,1970-01-01 04:30:00,78,7:00-8:00,290,3.5,3.5,8:30-10:00,1970-01-01 10:00:00,60,1970-01-01 12:00:00,40,13:30-15:00,15:00-17:00,,800,0.15,400,0.902


In [21]:
feature_name = [i for i in train.columns if i not in ['id','rate']]
x = feature_name
y = 'rate'

In [22]:
feature_name

['A1',
 'A2',
 'A3',
 'A4',
 'A5',
 'A6',
 'A7',
 'A8',
 'A9',
 'A10',
 'A11',
 'A12',
 'A13',
 'A14',
 'A15',
 'A16',
 'A17',
 'A18',
 'A19',
 'A20',
 'A21',
 'A22',
 'A23',
 'A24',
 'A25',
 'A26',
 'A27',
 'A28',
 'B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9',
 'B10',
 'B11',
 'B12',
 'B13',
 'B14']

In [23]:
aml = H2OAutoML(max_models=320, seed=2019, max_runtime_secs=12800)
aml.train(x=feature_name, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [27]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_BestOfFamily_AutoML_20190109_193515,0.000300638,0.0173389,0.000300638,0.0113838,0.00923033
XRT_1_AutoML_20190109_193515,0.000323951,0.0179986,0.000323951,0.0118992,0.00956932
DeepLearning_grid_1_AutoML_20190109_193515_model_17,0.000337638,0.0183749,0.000337638,0.0120108,0.00975058
GBM_1_AutoML_20190109_193515,0.00035548,0.0188542,0.00035548,0.0122279,0.0100201
GBM_2_AutoML_20190109_193515,0.000360846,0.0189959,0.000360846,0.012402,0.010091
GBM_grid_1_AutoML_20190109_193515_model_1,0.000361447,0.0190118,0.000361447,0.0124876,0.0100999
GBM_grid_1_AutoML_20190109_193515_model_9,0.000363868,0.0190753,0.000363868,0.0125205,0.0101312
DeepLearning_grid_1_AutoML_20190109_193515_model_13,0.000364602,0.0190946,0.000364602,0.012826,0.0101228
GBM_4_AutoML_20190109_193515,0.000364617,0.0190949,0.000364617,0.0124025,0.010141
DRF_1_AutoML_20190109_193515,0.000366558,0.0191457,0.000366558,0.0122915,0.0101711


In [32]:
automl_predictions = aml.predict(test).as_data_frame().values.flatten()

stackedensemble prediction progress: | (failed)


OSError: Job with key $03017f00000132d4ffffffff$_9eab3cc1a9f1e315c55f0b25643f4b36 failed with an exception: DistributedException from /127.0.0.1:54321: 'Categorical value out of bounds, got 186, next cat starts at 180', caused by java.lang.AssertionError: Categorical value out of bounds, got 186, next cat starts at 180
stacktrace: 
DistributedException from /127.0.0.1:54321: 'Categorical value out of bounds, got 186, next cat starts at 180', caused by java.lang.AssertionError: Categorical value out of bounds, got 186, next cat starts at 180
	at water.MRTask.getResult(MRTask.java:478)
	at water.MRTask.getResult(MRTask.java:486)
	at water.MRTask.doAll(MRTask.java:390)
	at water.MRTask.doAll(MRTask.java:396)
	at hex.StackedEnsembleModel.predictScoreImpl(StackedEnsembleModel.java:161)
	at hex.Model.score(Model.java:1357)
	at water.api.ModelMetricsHandler$1.compute2(ModelMetricsHandler.java:374)
	at water.H2O$H2OCountedCompleter.compute(H2O.java:1395)
	at jsr166y.CountedCompleter.e

In [36]:
automl_predictions = aml.predict(test)

stackedensemble prediction progress: | (failed)


OSError: Job with key $03017f00000132d4ffffffff$_9caa26a7493fd4342eafe0e70b3448ed failed with an exception: DistributedException from /127.0.0.1:54321: 'Categorical value out of bounds, got 186, next cat starts at 180', caused by java.lang.AssertionError: Categorical value out of bounds, got 186, next cat starts at 180
stacktrace: 
DistributedException from /127.0.0.1:54321: 'Categorical value out of bounds, got 186, next cat starts at 180', caused by java.lang.AssertionError: Categorical value out of bounds, got 186, next cat starts at 180
	at water.MRTask.getResult(MRTask.java:478)
	at water.MRTask.getResult(MRTask.java:486)
	at water.MRTask.doAll(MRTask.java:390)
	at water.MRTask.doAll(MRTask.java:396)
	at hex.StackedEnsembleModel.predictScoreImpl(StackedEnsembleModel.java:161)
	at hex.Model.score(Model.java:1357)
	at water.api.ModelMetricsHandler$1.compute2(ModelMetricsHandler.java:374)
	at water.H2O$H2OCountedCompleter.compute(H2O.java:1395)
	at jsr166y.CountedCompleter.e